In [12]:
# download input files

# Hadoop
(and Hadoop streaming)

In [1]:
#add logo

The map/reduce is a completely different paradigm 
Solving a certain subset of parallelizable problems 
around the bottleneck of ingesting input data from disk
Traditional parallelism brings the data to the computing machine
Map/reduce does the opposite, it brings the compute to the data
Input data is not stored on a separate storage system
Data exists in little pieces and is permanently stored on each computing node

HDFS is fundamental to Hadoop 
it provides the data chunking 
distribution across compute elements 
necessary for map/reduce applications to be efficient

# Word count
The '`Hello World`' for MapReduce

Among the simplest of full Hadoop jobs you can run

**IMAGE**

The MAP step will take the raw text and convert it to key/value pairs
Each key is a word
All keys (words) will have a value of 1
The REDUCE step will combine all duplicate keys 
By adding up their values (sum)
Every key (word) has a value of 1 (Map)
Output is reduced to a list of unique keys
Each key’s value corresponding to key's (word's) count.

Map function : processes data and generates a set of  intermediate key/value pairs.
Reduce function : merges all intermediate values  associated with the same intermediate key.

**IMAGE**

## A WordCount example 
*(with Java)*

Consider doing a word count of the following file 
using  MapReduce:

```
$ cat file.txt

Hello World Bye World
Hello Hadoop Goodbye Hadoop
```

The map function reads in words 
one at a time 
outputs (“word”, 1) for each parsed input word

Map output is:
```
(Hello, 1)
(World, 1)
(Bye, 1)
(World, 1)
(Hello, 1)
(Hadoop, 1)
(Goodbye, 1)
(Hadoop, 1)
```

The shuffle phase between map and reduce 
creates a  list of values associated with each key

Shuffle output / Reduce input is:

```
(Bye, (1))
(Goodbye, (1))
(Hadoop, (1, 1))
(Hello, (1, 1))
(World, (1, 1))
```

The reduce function sums the numbers in the list 
for each  key and outputs (word, count) pairs

Reduce output is: 
(also the output of the  MapReduce job)

```
(Bye, 1)
(Goodbye, 1)
(Hadoop, 2)
(Hello, 2)
(World, 2)
```

``` Java
package org.myorg;
import java.io.IOException;
import java.util.*;
import org.apache.hadoop.*

public class WordCount {

  public static class Map extends MapReduceBase implements Mapper<LongWritable, Text, Text, IntWritable> {
    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(LongWritable key, Text value, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {
      String line = value.toString();
      StringTokenizer tokenizer = new StringTokenizer(line);
      while (tokenizer.hasMoreTokens()) {
        word.set(tokenizer.nextToken());
        output.collect(word, one);
      }
    }
  }
    
  public static class Reduce extends MapReduceBase implements Reducer<Text, IntWritable, Text, IntWritable> {
    public void reduce(Text key, Iterator<IntWritable> values, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {
      int sum = 0;
      while (values.hasNext()) {
        sum += values.next().get();
      }
      output.collect(key, new IntWritable(sum));
    }
  }
    
  
  public static void main(String[] args) throws Exception {
    JobConf conf = new JobConf(WordCount.class);
    conf.setJobName("wordcount");

    conf.setOutputKeyClass(Text.class);
    conf.setOutputValueClass(IntWritable.class);

    conf.setMapperClass(Map.class);
    conf.setCombinerClass(Reduce.class);
    conf.setReducerClass(Reduce.class);

    conf.setInputFormat(TextInputFormat.class);
    conf.setOutputFormat(TextOutputFormat.class);

    FileInputFormat.setInputPaths(conf, new Path(args[0]));
    FileOutputFormat.setOutputPath(conf, new Path(args[1]));

    JobClient.runJob(conf);
  }
}
```

```
$ hadoop jar hadoop-examples.jar wordcount dft dft-output

10/03/16 11:40:51 INFO mapred.FileInputFormat: Total input paths to process : 1 10/03/16 11:40:51 INFO mapred.JobClient: Running job: job_201003161102_0002 10/03/16 11:40:52 INFO mapred.JobClient: map 0% reduce 0%
10/03/16 11:40:55 INFO mapred.JobClient: map 9% reduce 0% 10/03/16 11:40:56 INFO mapred.JobClient: map 27% reduce 0%
10/03/16 11:40:58 INFO mapred.JobClient: map 45% reduce 0% 10/03/16 11:40:59 INFO mapred.JobClient: map 81% reduce 0% 

10/03/16 11:41:01 INFO mapred.JobClient: map 100% reduce 0% 10/03/16 11:41:09 INFO mapred.JobClient: Job complete: job_201003161102_0002
```

```
$ hadoop dfs -cat dft-output/part-00000 | less 

“Information" 1 
"J" 1 
“Plain" 2 
“Project" 5 
```

**IMAGE**: cutting sandwiches with hadoop

# Exercise

Write a Java class to count letters inside a text file

*...just kidding!*

# Hadoop Streaming

```
head -2000 data/ngs/input.sam | tail -n 10 \
| awk '{ print $3":"$4 }' | sort | uniq -c

      3 chr1:142803456
      3 chr1:142803458
      1 chr1:142803465
      1 chr1:142803470
      2 chr1:142803471
```

In [1]:
myfile = '/data/worker/books/pg1232.txt'
%env myfile $myfile

env: myfile=/data/worker/books/pg1232.txt


In [8]:
%%bash
head -2000 $myfile | tail -n 10 | awk '{ print $3":"$4 }' | sort | uniq -c

      1 :
      1 and:turned
      1 are:found
      1 is:gained
      1 mercenaries,:when
      1 needed:to
      1 than:to
      1 their:head,
      1 valour.:The
      1 you.:In


Splitting the command:

1. `head -2000 data/ngs/input.sam | tail -n 10`

1. `awk '{ print $3":"$4 }’`

1. `sort`

1. `uniq -c`


**INPUT STREAM**
`head -2000 data/ngs/input.sam | tail -n 10`

**MAPPER**
`awk '{ print $3":"$4 }'`

**SHUFFLE**
`sort`

**REDUCER**
`uniq -c`

**OUTPUT STREAM**
`<STDOUT>`

```
# head -2000 data/ngs/input.sam | tail -n 10
HSCAN:421:C47DAACXX:3:1206:5660:99605	99	chr1	142803456	10	75M	=	142803517	136	CTGTGCATTCTTATGATTTTAATATTCTGTACATTTATTATTGATTTAAAATGCATTTTACCTTTTTCTTTAATA	@@CDDDD?DBFHFHIHIJJJJGHEIFHIIIHGDFEGEIAHAHGIIGIIIC?CBFCGHFCEG?@DGIIIGHIGGHC	X0:i:3	X1:i:1	XA:Z:chr21,+9746045,75M,0;chr1,+143355186,75M,0;chr1,-143233123,75M,1;	MD:Z:75	RG:Z:1	XG:i:0	AM:i:0	NM:i:0	SM:i:0	XM:i:0	XO:i:0	MQ:i:18	XT:A:R

[…]

# head -2000 data/ngs/input.sam | tail -n 10 | awk '{ print $3":"$4 }'
chr1:142803471
chr1:142803456
chr1:142803465
chr1:142803458
chr1:142803456
chr1:142803471
chr1:142803458
chr1:142803456
chr1:142803470
chr1:142803458

# head -2000 data/ngs/input.sam | tail -n 10 | awk '{ print $3":"$4 }' | sort
chr1:142803456
chr1:142803456
chr1:142803456
chr1:142803458
chr1:142803458
chr1:142803458
chr1:142803465
chr1:142803470
chr1:142803471
chr1:142803471

# head -2000 data/ngs/input.sam | tail -n 10 | awk '{ print $3":"$4 }' | sort | uniq -c
      3 chr1:142803456
      3 chr1:142803458
      1 chr1:142803465
      1 chr1:142803470
      2 chr1:142803471
```

### Considerations with bash pipes as simulation of MapReduce

* Serial steps
* No file distribution
* Single node
* Single mapper
* Single reducer
* Can we add a Combiner?

# Hadoop streaming
### Concepts and mechanisms

Hadoop streaming is a utility 
comes with the Hadoop distribution
Allows to create and run Map/Reduce jobs 
with any executable or script as the mapper and/or the reducer
Protocol steps:
Create a Map/Reduce job
Submit the job to an appropriate cluster
Monitor the progress of the job until it completes
Links to Hadoop HDFS job directories

### Why?
One of the unappetizing aspects of Hadoop to users of traditional HPC is that it is written in Java. 
Java is not originally designed to be a high-performance language
Learning it is not easy for domain scientists
 Hadoop allows you to write map/reduce code in any language you want using the Hadoop Streaming interface
It means turning an existing Python or Perl script into a Hadoop job
Does not require learning any Java at all

### MapReduce with binaries/executables
Executable is specified for mappers and reducers
each mapper task will launch the executable as a separate process 
converts inputs into lines and feed to the stdin of the process
the mapper collects the line oriented outputs from the stdout of the process 
converts each line into a key/value pair
By default, the prefix of a line up to the first tab character is the key and the rest of the line (excluding the tab character) will be the value
e.g. ”this is the key\tvalue is the rest\n”
If there is no tab character in the line, then entire line is considered as key and the value is null (!)

A streaming command line example:
```
$ hadoop jar $HADOOP_HOME/hadoop-streaming.jar \
    -input myInputDirs \
    -output myOutputDir \
    -mapper org.apache.hadoop.mapred.lib.IdentityMapper \
    -reducer /bin/wc
```

A streaming command line example **for python**:
```
$ hadoop jar hadoop-streaming-1.2.1.jar \
    -input input_dir/ \
    -output output_dir/ \
    -mapper mapper.py \
    -file mapper.py \
    -reducer reducer.py \
    -file reducer.py 

```

Before submitting the Hadoop Streaming job:

* Make sure your scripts have no errors
* Do mapper and reducer scripts actually work?

This is just a matter of running them through pipes on a **little bit** of sample data,

like `cat` or `head` linux bash commands, with pipes, as seen before.

```
# Simulating hadoop streaming with bash pipes
$ cat $file | python mapper.py | sort | python reducer.py
```

First approach: split

In [15]:
%%writefile mapper.py
# -*- coding: utf-8 -*-
import sys

for line in sys.stdin:
    line = line.strip()
    pieces = line.split('\t')

print(pieces) 
#prints [“piece1”,”piece2”…]

Overwriting mapper.py


In [2]:
! head -n 10 $myfile | python mapper.py 
# | sort | python reducer.py

Traceback (most recent call last):
  File "mapper.py", line 4, in <module>
    for line in sys.stdin:
  File "/opt/conda/lib/python3.4/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xef in position 0: ordinal not in range(128)


In [15]:
%%writefile mapper.py

# -*- coding: utf-8 -*-
TAB = "\t"
import sys

# Cycle current streaming data
for line in sys.stdin:

    # Clean input
    line = line.strip()
    # Skip SAM/BAM headers
    if line[0] == "@":
        continue

    # Use data
    pieces = line.split(TAB)
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]
    print mychr,mystart.__str__()
    sys.exit(1)

Overwriting mapper.py


```
#cat data/ngs/input.sam \ 
| python ngs/hs/hsmapper.py

chrM 14
```

In [ ]:
%%writefile mapper.py

# -*- coding: utf-8 -*-
TAB = "\t"
import sys

# Cycle current streaming data
for line in sys.stdin:

    # Use data
    pieces = line.split(TAB)
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]

    mystop = mystart + len(myseq)

    # Each element with coverage
    for i in range(mystart,mystop):
        results = [mychr+SEP+i.__str__(), "1"]
        print(TAB.join(results))
        sys.exit(1)


```
root@5a57dab12438:/course-exercises

#cat data/ngs/input.sam \ 
| python ngs/hs/hsmapper.py

chrM 14 1

[…]

chrM:79	1
chrM:80	1
chrM:81	1
chrM:82	1
chrM:83	1
chrM:84	1
chrM:85	1
chrM:86	1
chrM:87	1
chrM:88	1
```

### Shuffle step 

A lot happens, transparent to the developer
Mappers’s output is transformed and distributed to the reducers
All key/value pairs are sorted before sent to reducer function
Pairs sharing the same key are sent to the same reducer
If you encounter a key that is different from the last key you processed, you know that previous key will never appear again
If your keys are all the same
only use one reducer and gain no parallelization
come up with a more unique key if this happens

### Reducer

In [ ]:
%%writefile reducer.py

# -*- coding: utf-8 -*-
import sys

# Cycle current streaming data
for line in sys.stdin:

    # Clean input
    line = line.strip()
    value, count = line.split(TAB)
    count = int(count)

    # if this is the first iteration
    if not last_value:
        last_value = value

    # if they're the same, log it
    if value == last_value:
        value_count += count
    else:
        # state change
        result = [last_value, value_count]
        print TAB.join(str(v) for v in result)
        last_value = value
        value_count = 1

# LAST ONE after all records have been received
print TAB.join(str(v) for v in [last_value, value_count])

```
root@5a57dab12438:/course-exercises

#cat data/ngs/input.sam \ 
| python ngs/hs/hsmapper.py \
| sort \
| python ngs/hs/hsreducer.py

chr10:103270024	1
chr10:103270025	1

[…]

chr21:48110960	2
chr21:48110961	3
chr21:48110962	3
chr21:48110963	6
chr21:48110964	6
chr21:48110965	6
chr21:48110966	6
chr21:48110967	8
chr21:48110968	8
chr21:48110969	11 

```

### Switching to real Hadoop

A working python code tested on pipes should work with Hadoop Streaming

To make this work we need to handle copy of input and output file 
inside the Hadoop FS
Also the job tracker logs will be found inside HDFS
We are going to build a bash script to make our workflow

In [ ]:
## Preprocessing

---

Final launch via bash command for using Hadoop streaming

---

## Final thoughts on Hadoop Streaming 

Provides options to write MapReduce jobs in other languages
One of the best examples of flexibility available to MapReduce
Fast
Simple
Close to the original standard Java API power

Even executables can be used to work as a MapReduce job (!)

Where it really works

When the developer do not have knowhow of Java 
Write Mapper/Reducer in any scripting language 
Faster

Cons

Force scripts in a Java VM
Although free overhead
The program/executable should be able to take input from STDIN and produce output at STDOUT
Restrictions on the input/output formats
Does not take care of input and output file and directory preparation
User have to implement hdfs commands “hand-made”

Where it falls short

No pythonic way to work the MapReduce code
Because it was not written specifically for python

Recap 

Hadoop streaming handles Hadoop in almost a classic manner
Wrap any executable (and script)
Wrap python scripts
Runnable on a cluster
using a non-interactive, all-encapsulated job

* Docker lightweight virtualization
* Two environments 
* Schema